In [1]:
!pip install cloudscraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from bs4 import BeautifulSoup
import cloudscraper
import csv
import re
import time
import pandas as pd

In [3]:
list_of_links = []

with open('links_file.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Url', 'Time posted', 'Date posted'])

for i in range(104, 5050, 1): 
    url      = f"https://www.nairaland.com/news/{i}" 
    scraper  = cloudscraper.create_scraper()
    raw_html = (scraper.get(url).text)
    soup     = BeautifulSoup(raw_html, "html.parser")
    post_elements = soup.select("hr + a") 

    for elem in post_elements: 
        post_link = elem["href"] 
        post_link = re.sub(r"/\d+(?=#|$)", "", post_link) 

        time_and_date = elem.find_next_siblings("b") 
        post_time     = time_and_date[0].text 
        post_date     = time_and_date[1].text
        
        if len(time_and_date) > 2 and time_and_date[2].text.isdigit():
            post_year = time_and_date[2].text
        else:
            post_year = '2023'

        list_of_links.append([post_link, post_time, f"{post_date}, {post_year}"]) 

    if i % 100 == 0 and i != 0:
         time.sleep(120)

    with open('links_file.csv', mode='a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)

        for item in list_of_links:
            csv_writer.writerow(item)

    list_of_links.clear()


In [4]:
#read from csv and convert to dataframe
links_file = pd.read_csv('links_file.csv')
#display the scrapped list of links
links_file

,Url,Time posted,Date posted
0,https://www.nairaland.com/7503295/pdp-g-5-govs...,10:39am,"Dec 31, 2022"
1,https://www.nairaland.com/7503173/suspected-ba...,10:38am,"Dec 31, 2022"
2,https://www.nairaland.com/7503347/9ice-wife-su...,10:38am,"Dec 31, 2022"
3,https://www.nairaland.com/7503670/onslaught-la...,10:35am,"Dec 31, 2022"
4,https://www.nairaland.com/7503259/man-beat-dau...,10:01am,"Dec 31, 2022"
...,...,...,...
257905,https://www.nairaland.com/nigeria/topic-837796...,6:12pm,"Jan 04, 2012"
257906,https://www.nairaland.com/nigeria/topic-838840...,5:59pm,"Jan 04, 2012"
257907,https://www.nairaland.com/nigeria/topic-838832...,5:47pm,"Jan 04, 2012"
257908,https://www.nairaland.com/nigeria/topic-838863...,5:40pm,"Jan 04, 2012"


In [ ]:
#

### SPLITTING THE URLS INTO 6 PARTS AND SAVING EACH INTO SEPARATE CSV FILE

In [5]:

# Define the number of rows per file
rows_per_file = 50000
# Calculate the number of full files needed
num_full_files = len(links_file) // rows_per_file

# Calculate the number of rows in the last file
remainder_rows = len(links_file) % rows_per_file

# Split the data and save it to separate files
for i in range(num_full_files):
    start = i * rows_per_file
    end = (i + 1) * rows_per_file
    links_file[start:end].to_csv(f'links_file_{i + 1}.csv', index=False)

# Save the remaining rows to the last file
if remainder_rows > 0:
    start = num_full_files * rows_per_file
    end = len(links_file)
    links_file[start:end].to_csv(f'links_file_{num_full_files + 1}.csv',  
    index=False)


In [6]:
links_file_1 = pd.read_csv('links_file_1.csv')
links_file_2 = pd.read_csv('links_file_2.csv')
links_file_3 = pd.read_csv('links_file_3.csv')
links_file_4 = pd.read_csv('links_file_4.csv')
links_file_5 = pd.read_csv('links_file_5.csv')
links_file_6 = pd.read_csv('links_file_6.csv')


In [9]:
print('Number of Urls in first part is', len(links_file_1))
print('Number of Urls in second part is', len(links_file_2))
print('Number of Urls in third part is', len(links_file_3))
print('Number of Urls in fourth part is', len(links_file_4))
print('Number of Urls in fifth part is', len(links_file_5))
print('Number of Urls in sixth part is', len(links_file_6))
print('*******************************************')
print('Total Number of Urls in all parts:', len(links_file))
print('*******************************************')


Number of Urls in first part is 50000
Number of Urls in second part is 50000
Number of Urls in third part is 50000
Number of Urls in fourth part is 50000
Number of Urls in fifth part is 50000
Number of Urls in sixth part is 7910
*******************************************
Total Number of Urls in all parts: 257910
*******************************************
